In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd
import os
import gff3

In [8]:
### your fasta sequence file
fasta_file = '/Users/andreas/Bacillus/Bioinformatics/FASTA/NC_000964v3.fasta'
cluster_file = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_19thJune2024/Kre_CRAC_analysis/4_pyClusterReads_delsOnly/Kre_merged.novo_count_output_cDNAs.gtf_merged'
clusters = pd.read_csv(cluster_file, sep = '\t', names = gff3.header, comment = '#')

In [9]:
### load fasta file
with open(fasta_file, 'r') as f:
    lines = f.readlines()
    lines = lines[1:]
    lines = [l.replace('\n', '') for l in lines]
    lines = ''.join(lines)

### function to reverse complement genes on reverse strand
def reverse_complement(sequence):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_seq = sequence[::-1]
    complement_seq = ''.join([complement_dict[base] for base in reverse_seq])
    return complement_seq

In [10]:
### make dataframe with fasta sequences for all clusters
fasta_sequences = []
coordinates = []
for index, row in clusters.iterrows():
    if row['strand'] == '+':
        fasta_sequences.append(lines[row['start'] - 1:row['end']])
    else:
        fasta_sequences.append(reverse_complement(lines[row['start'] - 1:row['end']]))
    coordinates.append(f"{row['start']}_{row['end']}_{row['strand']}")

file_name = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_19thJune2024/kre_ALL_dels_clusters_fasta.txt'
with open(file_name, 'w') as f:
    for coord, seq in zip(coordinates, fasta_sequences):
            f.write(f">{coord}\n")
            f.write(f"{seq}\n")

In [14]:
### Trim fasta sequnces to retain only middle 50 bp
def trim_fasta_middle(input_filename, output_filename):
    def get_middle_50bp(sequence):
        length = len(sequence)
        if length <= 50:
            return sequence
        start = (length - 50) // 2
        return sequence[start:start + 50]

    with open(input_filename, 'r') as infile, open(output_filename, 'w') as outfile:
        sequence_id = ''
        sequence = ''
        for line in infile:
            if line.startswith('>'):
                if sequence:
                    trimmed_sequence = get_middle_50bp(sequence)
                    outfile.write(f'{sequence_id}\n{trimmed_sequence}\n')
                sequence_id = line.strip()
                sequence = ''
            else:
                sequence += line.strip()
        # Write the last sequence
        if sequence:
            trimmed_sequence = get_middle_50bp(sequence)
            outfile.write(f'{sequence_id}\n{trimmed_sequence}\n')

# Example usage
input_filename = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_19thJune2024/khpA_ALL_dels_clusters_fasta.txt'
output_filename = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_19thJune2024/khpA_ALL_dels_clusters_fasta_trimmed50.txt'

trim_fasta_middle(input_filename, output_filename)
